# Setting Up All Artifacts details

In [ ]:
## Give appropriate permission to the directory "FOLDER_WITH_ARTIFACTS" you are working with
import os
os.environ['SNPE_ROOT']="/local/mnt/workspace/snpe/2.16.0.231029/"#set up your snpe path here.
os.environ['RAW_FILE_FOLDER']="input/raw"
os.environ['DLC32']="models/srgan_fp32.dlc"
os.environ['DLC8']="models/srgan_w8a8.dlc"
os.environ['TARGET_INPUT_LIST']="input/input.txt"
os.environ['ONDEVICE_FOLDER']="srgan"
os.environ['DEVICE_HOST']="localhost"
os.environ['DEVICE_ID']="2dce6316" #fill your device-id. Use command "adb devices" to get devices names. example :"e18d5d0"
os.environ['SNPE_TARGET_ARCH']="aarch64-android"
os.environ['SNPE_TARGET_STL']="libc++_shared.so"

In [ ]:
%%bash
git clone https://github.com/andreas128/mmsr.git

In [ ]:
%%bash
cd mmsr 
git reset --hard a73b318f0f07feb6505ef5cb1abf0db33e33807a

In [ ]:
%%bash
cd mmsr
git apply ../mmsr-changes.patch

In [ ]:
%%bash
git clone https://github.com/quic/aimet-model-zoo/
cp -r srgan.patch aimet-model-zoo
cd aimet-model-zoo
git apply srgan.patch

## Steps to generate ONNX model

In [ ]:
%%bash
cp -r aimet-model-zoo/aimet_zoo_torch/srgan/evaluators/srgan_quanteval.py mmsr/
rm -rf aimet-model-zoo/

In [ ]:
%%bash
cd mmsr/codes/
touch __init__.py


In [ ]:
%%bash
cd mmsr/
python srgan_quanteval.py --mmsr-path '../mmsr'

In [ ]:
import cv2
import glob
import os
import numpy as np
import torch
from math import ceil, floor


In [ ]:
os.makedirs('models', exist_ok= True)
os.makedirs('output', exist_ok= True)

In [ ]:
%%bash
cp -r mmsr/srgan.onnx models/

In [ ]:
%%bash
source $SNPE_ROOT/bin/envsetup.sh
snpe-onnx-to-dlc --input_network models/srgan.onnx --output_path models/srgan_fp32.dlc


# Download dataset
<ul>
    <li>  Dataset link wget https://figshare.com/ndownloader/files/38256855  </li>
<li>Below block will automatically download datsest, but in case if it fails please download from above link.</li>
    <li> Recommended, to comment below code, if already downloaded dataset once.</li>
    <ul>

In [ ]:
%%bash
wget https://figshare.com/ndownloader/files/38256855
unzip 38256855 -d input
rm -rf 38256855
rm -rf input/Set14/image_SRF_4
rm -rf input/Set14/image_SRF_3
mkdir input/raw
find input/Set14/image_SRF_2 -name '*_LR*' -delete
mv input/Set14/image_SRF_2/* input/Set14/
rm -rf input/Set14/image_SRF_2/

# Pre-processing data

In [ ]:
img_paths =  glob.glob(os.path.join("input/Set14/", '*'))
img_paths = sorted(img_paths)
img_paths

In [ ]:
import requests

url = 'https://raw.githubusercontent.com/quic/aimet-model-zoo/develop/aimet_zoo_torch/common/super_resolution/imresize.py'
response = requests.get(url)

if response.status_code == 200:
    with open('imresize.py', 'wb') as f:
        f.write(response.content)
        print('done')
else:
    print("error")

In [ ]:
RGB_WEIGHTS = torch.FloatTensor([65.481, 128.553, 24.966])
from imresize import *
def preprocess(img, scaling_factor=2):
    lr_img, hr_img = create_hr_lr_pair(img, scaling_factor)

    return lr_img, hr_img

In [ ]:
def create_hr_lr_pair(img, scaling_factor=2):
    height, width = img.shape[0:2]

    # Take the largest possible center-crop of it such that its dimensions are perfectly divisible by the scaling factor
    x_remainder = width % (scaling_factor)
    y_remainder = height % (scaling_factor)
    left = x_remainder // 2
    top = y_remainder // 2
    right = left + (width - x_remainder)
    bottom = top + (height - y_remainder)
    hr_img = img[top:bottom, left:right]

    hr_height, hr_width = hr_img.shape[0:2]

    hr_img = np.array(hr_img, dtype='float32')
    lr_img = imresize(hr_img, 1. / scaling_factor)  # equivalent to matlab's imresize
    flag=0
    lr_img = np.uint8(np.clip(lr_img, 0., 255.))  # this is to simulate matlab's imwrite operation
    hr_img = np.uint8(hr_img)
    lr_height, lr_width = lr_img.shape[0:2]

    # Sanity check
    assert hr_width == lr_width * scaling_factor and hr_height == lr_height * scaling_factor
    lr_img = convert_image(lr_img, source='array', target='[0, 1]')
    hr_img = convert_image(hr_img, source='array', target='[0, 1]')

    return lr_img, hr_img

In [ ]:
def convert_image(img, source, target):
    if source == 'array':
        img = torch.from_numpy(img.transpose((2, 0, 1))).contiguous()#chw
        img = img.to(dtype=torch.float32).div(255)
    elif source == '[0, 1]':
        img = torch.clamp(img, 0, 1)  # useful to post-process output of models that can overspill
    
    if target == '[0, 1]':
        pass  # already in [0, 1]
    elif target == 'y-channel':
        img = torch.matmul(img.permute(0, 2, 3, 1), RGB_WEIGHTS.to(img.device)) + 16.
    
    return img

In [ ]:
def post_process(img):
    img = img.detach().cpu().numpy()
    img = np.clip(255. * img, 0., 255.)
    img = np.uint8(img)
    img = img.transpose(1, 2, 0)#hwc
    return img


In [ ]:
def load_dataset(test_images_dir, scaling_factor=2):
    # Input images for the model
    INPUTS_LR = []

    # Post-processed images for visualization
    IMAGES_LR = []
    IMAGES_HR = []
    
    # Load the test images
    count=0
    img_paths =  glob.glob(os.path.join(test_images_dir, '*'))
    img_paths = sorted(img_paths)
    for img_path in img_paths:
        img = cv2.resize(cv2.imread(img_path),[512,512],interpolation=cv2.INTER_CUBIC)
        img = cv2.cvtColor(img, cv2.COLOR_BGR2RGB)
        lr_img, hr_img = preprocess(img, scaling_factor)#chw
        INPUTS_LR.append(lr_img)#chw
        IMAGES_LR.append(post_process(lr_img))#hwc
        IMAGES_HR.append(post_process(hr_img))#hwc

    return INPUTS_LR, IMAGES_LR, IMAGES_HR

In [ ]:
test_images_dir = "input/Set14"
INPUTS_LR, IMAGES_LR, IMAGES_HR = load_dataset(test_images_dir, scaling_factor=4)
for i, img_lr in enumerate(INPUTS_LR):
    img_lr = img_lr.cpu().detach().numpy()
    img_lr = img_lr.astype(np.float32)
    fid = open("input/raw/img"+str(i)+ ".raw", 'wb')
    img_lr.tofile(fid)
    fid.close()

In [ ]:
import matplotlib.pyplot as plt
plt.imshow(IMAGES_LR[0])
plt.imshow(IMAGES_HR[0])

In [ ]:
with open("input/input.txt", "w") as f:
    for i in range(14):
        file_path = f"./raw/img{i}.raw"
        f.write(file_path + "\n")

In [ ]:
%%bash
source $SNPE_ROOT/bin/envsetup.sh
cd input
snpe-dlc-quantize --input_dlc ../models/srgan_fp32.dlc --input_list input.txt --use_enhanced_quantizer --use_adjusted_weights_quantizer --axis_quant --output_dlc ../models/srgan_w8a8.dlc

# Post-process model output

In [ ]:
def post_process_sr(img):
#     img = img.detach().cpu().numpy()
    img = np.fromfile(img, np.float32)
    img = img.reshape((3,512, 512)).astype(np.float32)
    img = np.clip(255. * img, 0., 255.)
    img = np.uint8(img)
    img = img.transpose(1, 2, 0)#hwc
    return img

Method to calcualte PSNR

In [ ]:
def compute_psnr(img_pred, img_true, data_range=255., eps=1e-8):
    """
    Compute PSNR between super-resolved and original images.
    
    :param img_pred:
        The super-resolved image obtained from the model
    :param img_true:
        The original high-res image
    :param data_range:
        Default = 255
    :param eps:
        Default = 1e-8
    :return:
        PSNR value
    """
    err = (img_pred - img_true) ** 2
    err = err.mean(dim=-1).mean(dim=-1)

    return 10. * torch.log10((data_range ** 2) / (err + eps))

In [ ]:
def evaluate_psnr(y_pred, y_true):
    """
    Evaluate individual PSNR metric for each super-res and actual high-res image-pair.
    
    :param y_pred:
        The super-resolved image from the model
    :param y_true:
        The original high-res image
    :return:
        The evaluated PSNR metric for the image-pair
    """
    y_pred = y_pred.transpose(2, 0, 1)[None] / 255.
    y_true = y_true.transpose(2, 0, 1)[None] / 255.

    sr_img = convert_image(torch.FloatTensor(y_pred),
                           source='[0, 1]',
                           target='y-channel')
    hr_img = convert_image(torch.FloatTensor(y_true),
                           source='[0, 1]',
                           target='y-channel')

    return compute_psnr(sr_img, hr_img)

In [ ]:
def evaluate_average_psnr(sr_images, hr_images):
    """
    Evaluate the avg PSNR metric for all test-set super-res and high-res images.

    :param sr_images:
        The list of super-resolved images obtained from the model for the given test-images
    :param hr_images:
        The list of original high-res test-images
    :return:
        Average PSNR metric for all super-resolved and high-res test-set image-pairs
    """
    psnr = []
    for sr_img, hr_img in zip(sr_images, hr_images):
        psnr.append(evaluate_psnr(sr_img, hr_img))
        print(evaluate_psnr(sr_img, hr_img))

 

    # Convert the list of tensor values to a tensor array
    psnr_tensor = torch.cat(psnr)

 

    # Calculate the mean of the tensor array
    average_psnr = torch.mean(psnr_tensor)

    return average_psnr

# Creating Bin and Lib Folder on Device 

In [ ]:
%%bash
export DEVICE_SHELL="adb -H $DEVICE_HOST -s $DEVICE_ID"
$DEVICE_SHELL shell "mkdir -p /data/local/tmp/snpeexample/$SNPE_TARGET_ARCH/bin" && $DEVICE_SHELL shell "mkdir -p /data/local/tmp/snpeexample/$SNPE_TARGET_ARCH/lib" && $DEVICE_SHELL shell "mkdir -p /data/local/tmp/snpeexample/dsp/lib"

# Pushing all Lib and Bin files onto Device

In [ ]:
%%bash
export DEVICE_SHELL="adb -H $DEVICE_HOST -s $DEVICE_ID"
$DEVICE_SHELL push $SNPE_ROOT/lib/$SNPE_TARGET_ARCH/$SNPE_TARGET_STL /data/local/tmp/snpeexample/$SNPE_TARGET_ARCH/lib
$DEVICE_SHELL push $SNPE_ROOT/bin/$SNPE_TARGET_ARCH/snpe-net-run /data/local/tmp/snpeexample/$SNPE_TARGET_ARCH/bin
$DEVICE_SHELL push $SNPE_ROOT/lib/hexagon-v75/unsigned/*.so /data/local/tmp/snpeexample/dsp/lib
$DEVICE_SHELL push $SNPE_ROOT/lib/$SNPE_TARGET_ARCH/*.so /data/local/tmp/snpeexample/$SNPE_TARGET_ARCH/lib

# Pushing Artifacts on to Device

In [ ]:
%%bash
export DEVICE_SHELL="adb -H $DEVICE_HOST -s $DEVICE_ID"
$DEVICE_SHELL shell "mkdir -p /data/local/tmp/$ONDEVICE_FOLDER"

In [ ]:
%%bash
#find ./raw -name *.raw > list.txt
export DEVICE_SHELL="adb -H $DEVICE_HOST -s $DEVICE_ID"
$DEVICE_SHELL push $DLC32 /data/local/tmp/$ONDEVICE_FOLDER
$DEVICE_SHELL push $DLC8 /data/local/tmp/$ONDEVICE_FOLDER
$DEVICE_SHELL push $RAW_FILE_FOLDER /data/local/tmp/$ONDEVICE_FOLDER
$DEVICE_SHELL push $TARGET_INPUT_LIST /data/local/tmp/$ONDEVICE_FOLDER

# Inferencing 8 bit DLC on DSP Runtime
Give name of DLC in OUTPUT_DLC_QUANTIZED8 and ondevice folder

In [ ]:
%%bash
export DEVICE_SHELL="adb -H $DEVICE_HOST -s $DEVICE_ID"
$DEVICE_SHELL shell
export LD_LIBRARY_PATH=$LD_LIBRARY_PATH:/data/local/tmp/snpeexample/aarch64-android/lib
export PATH=$PATH:/data/local/tmp/snpeexample/aarch64-android/bin
export ADSP_LIBRARY_PATH="/data/local/tmp/snpeexample/dsp/lib;/system/lib/rfsa/adsp;/system/vendor/lib/rfsa/adsp;/dsp"
export OUTPUT_FOLDER=OUTPUT_8b_DSP
export OUTPUT_DLC_QUANTIZED8=srgan_w8a8.dlc
export ONDEVICE_FOLDER="srgan"
cd /data/local/tmp/$ONDEVICE_FOLDER &&
snpe-net-run --container $OUTPUT_DLC_QUANTIZED8 --input_list input.txt --output_dir $OUTPUT_FOLDER --use_dsp

# Inferencing 32b DLC on CPU Runtime
Give name of DLC in OUTPUT_DLC_32 and ondevice folder

In [ ]:
%%bash
export DEVICE_SHELL="adb -H $DEVICE_HOST -s $DEVICE_ID"
$DEVICE_SHELL shell
export LD_LIBRARY_PATH=$LD_LIBRARY_PATH:/data/local/tmp/snpeexample/aarch64-android/lib
export PATH=$PATH:/data/local/tmp/snpeexample/aarch64-android/bin
export OUTPUT_FOLDER=OUTPUT_32b_CPU
export OUTPUT_DLC_32=srgan_fp32.dlc
export ONDEVICE_FOLDER="srgan"
cd /data/local/tmp/$ONDEVICE_FOLDER &&
snpe-net-run --container $OUTPUT_DLC_32 --input_list input.txt --output_dir $OUTPUT_FOLDER

# Pulling Output folder generated on different precision and cores

In [ ]:
%%bash
rm -rf output/OUTPUT_8b_DSP
rm -rf output/OUTPUT_32b_CPU


In [ ]:
%%bash

export DEVICE_SHELL="adb -H $DEVICE_HOST -s $DEVICE_ID"
# $DEVICE_SHELL pull /data/local/tmp/$ONDEVICE_FOLDER/OUTPUT_8b_DSP output/OUTPUT_8b_DSP
$DEVICE_SHELL pull /data/local/tmp/$ONDEVICE_FOLDER/OUTPUT_32b_CPU output/OUTPUT_32b_CPU

# Calculate PSNR
* Pass path of two raw image in Argument.

In [ ]:
val = []
for i in range(10):
    val.append(IMAGES_HR[i])
val[0].shape

In [ ]:
folder = ["output/OUTPUT_32b_CPU"]
RGB_WEIGHTS = torch.FloatTensor([65.481, 128.553, 24.966])
for j in range(0,len(folder)):
    IMAGES_SR = []
    for i in range(0,13):
        IMAGES_SR.append(post_process_sr(folder[j]+"/Result_"+str(i)+"/155.raw"))
    print(folder[j]," (Average PSNR) :: ",evaluate_average_psnr(IMAGES_SR, IMAGES_HR))
    print("\n============================\n")

In [ ]:
import matplotlib.pyplot as plt
plt.imshow(IMAGES_SR[5])

In [ ]:
plt.imshow(IMAGES_HR[5])